In [27]:
import pandas as pd
from pycelonis import get_celonis  # Stellen Sie sicher, dass Sie die richtige Import-Anweisung verwenden
from pycelonis_core.utils.errors import PyCelonisNotFoundError
import json

# Laden und Vorverarbeiten der Daten
AIRLINE_CODE = "ZYXW"
df_airline = pd.read_parquet(f"./{AIRLINE_CODE}_tripfiles_conv.parquet", engine="pyarrow")

# Erstellen der flightid-Spalte
df_airline["flightid"] = df_airline.apply(
    lambda x: f"{x['airline_code']}_{x['flight_number']}_{x['flight_date']}_{x['departure_airport']}",
    axis=1,
)

# Entfernen von Duplikaten
df_airline = df_airline.drop_duplicates(subset=[col for col in df_airline.columns if col != "id"])

# Entfernen unnötiger Spalten
df_airline.drop(
    columns=[
        "id",
        "airline_code",
        "flight_number",
        "flight_date",
        "departure_airport",
        "header_line",
        "entry_details",
    ],
    inplace=True,
)

# Konvertieren der creation_time in ein Datetime-Objekt
df_airline["creation_time"] = pd.to_datetime(df_airline["creation_time"])

output_csv_path = f"./{AIRLINE_CODE}_tripfiles_conv.csv"
df_airline.to_csv(output_csv_path, index=False)